# MODIS Data + WXEE

In [89]:
import ee
import wxee
import eemont

In [2]:
ee.Authenticate()

Unable to connect to VS Code server: Error in request.
Error: connect ENOENT /run/user/1026/vscode-ipc-fa5f05c5-8f9f-49e2-bf61-cf8fa329cc1c.sock
    at PipeConnectWrap.afterConnect [as oncomplete] (node:net:1494:16) {
  errno: -2,
  code: 'ENOENT',
  syscall: 'connect',
  address: '/run/user/1026/vscode-ipc-fa5f05c5-8f9f-49e2-bf61-cf8fa329cc1c.sock'
}


Enter verification code:  4/1AfJohXnRkEPPzCT5yEECrk9vngkRD6OxBLnkfHUuroVuA07WqzP6EvTNXbk



Successfully saved authorization token.


In [90]:
wxee.Initialize()

* MODIS NADIR BRDF-Adjusted Reflectance Daily - [GEE Catalogue](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD43A4)
* MODIS Terra Surface Reflanctance Daily Global 250m [GEE Catalogue](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD09GQ)
* **MODIS Terra Surface Reflectance Daily Global 1km and 500km** - [GEE Catalogue](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD09GA)
   * `MODIS/061/MOD09GA`

In [220]:
# image collection
image_collection = "MODIS/061/MOD09GA"
collection = wxee.TimeSeries(image_collection)

# choose the date
start_date = "2020-10-01"
end_date = "2020-11-01"
collection = collection.filterDate(start_date, end_date)

# mask clouds
method = "cloud_prob"
collection = collection.maskClouds(method=method, maskShadows=False)

# scale + offset
collection = collection.scaleAndOffset()

# select bands
variables = [f"sur_refl_b0{i}" for i in range(1,7+1)]
collection = collection.select(selectors=variables)

In [ ]:
# Spatial Resolution in CRS Units [m]
scale = 10e3

# The coordinate reference system to use (NAD83 Albers CONUS)
crs = "EPSG:4326" 

# region
countries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")
spain = countries.filterMetadata("ADM0_NAME", "equals", "Spain")
region = spain.geometry().bounds()

### Load Data Into Xarray

In [221]:
ds = collection.wx.to_xarray(
    region=region, 
    scale=scale, 
    crs=crs,
)

Requesting data: 100%|██████████████████████████| 31/31 [00:10<00:00,  2.89it/s]


In [222]:
ds

<xarray.Dataset>
Dimensions:       (time: 31, x: 252, y: 181)
Coordinates:
  * time          (time) datetime64[ns] 2019-10-01 2019-10-02 ... 2019-10-31
  * x             (x) float64 -18.19 -18.1 -18.01 -17.92 ... 4.177 4.267 4.357
  * y             (y) float64 43.79 43.7 43.61 43.52 ... 27.89 27.8 27.71 27.62
    spatial_ref   int64 0
Data variables:
    sur_refl_b01  (time, y, x) float64 nan nan nan nan ... 0.3863 0.3935 0.4025
    sur_refl_b02  (time, y, x) float64 nan nan nan nan ... 0.4711 0.4776 0.4852
    sur_refl_b03  (time, y, x) float64 nan nan nan nan ... 0.172 0.1761 0.1844
    sur_refl_b04  (time, y, x) float64 nan nan nan nan ... 0.2742 0.2801 0.2893
    sur_refl_b05  (time, y, x) float64 nan nan nan nan ... 0.5573 0.5621 0.5659
    sur_refl_b06  (time, y, x) float64 nan nan nan nan ... 0.5888 0.5935 0.5978
Attributes:
    AREA_OR_POINT:           Area
    TIFFTAG_RESOLUTIONUNIT:  1 (unitless)
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    _FillValue:              -32768.0
    scale_factor:            1.0
    add_offset:              0.0

In [154]:
from operator import itemgetter, methodcaller
import hydra_zen
import toolz

init_partial_fn = hydra_zen.make_custom_builds_fn(zen_partial=True, )
init_fn = hydra_zen.make_custom_builds_fn(zen_partial=False)

In [193]:
spain.geometry().bounds()

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.bounds",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "Collection.geometry",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.loadTable",
                      "arguments": {
                        "tableId": {
                          "constantValue": "FAO/GAUL_SIMPLIFIED_500m/2015/level0"
                        }
                      }
                    }
                  },
                  "filter": {
                    "functionInvocationValue": {
                      "functionName": "Filter.equals",
                      "arguments": {
                        "leftField": {
       

In [194]:
data_retrieval = hydra_zen.make_config(
    zen_dataclass={'cls_name': 'load_modis'},
    # 1 - filter dates
    # _0=init_partial_fn(toolz.identity),
    _1=init_partial_fn(ee.ImageCollection, image_collection),
    _2=init_fn(methodcaller, "filterDate", start_date, end_date),
    #  MASK CLOUDS
    _3=init_fn(
    methodcaller, "maskClouds", method="cloud_prob", maskShadows=False),
    _4=init_fn(methodcaller, "scaleAndOffset"),
    # _5=init_fn(
    #     methodcaller, "wx.to_xarray",
    #     region=init_fn(
    #         ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0"),
    #         countries.filterMetadata("ADM0_NAME", "equals", "Spain")), 
    #     scale=scale, 
    #     crs=crs
    # )
)

In [195]:
config = hydra_zen.instantiate(data_retrieval)
config

{'_1': functools.partial(<class 'ee.imagecollection.ImageCollection'>, 'MODIS/061/MOD09GA'), '_2': operator.methodcaller('filterDate', '2019-10-01', '2019-11-01'), '_3': operator.methodcaller('maskClouds', method='cloud_prob', maskShadows=False), '_4': operator.methodcaller('scaleAndOffset')}

In [189]:
# save to a yaml file
config_path = "./config.yaml"
hydra_zen.save_as_yaml(data_retrieval, f=config_path)

In [190]:
def zen_compose(d):
    return toolz.compose_left(*(d[k] for k in sorted(d)))

In [191]:
# initialize config
config = hydra_zen.load_from_yaml(config_path)


# load config
fn_asdicts = hydra_zen.instantiate(config)

fn = zen_compose(fn_asdicts)

collection = ee.ImageCollection(image_collection)
out = fn()
out

In [ ]:
t0 

base_prepro = hydra_zen.make_config(
    zen_dataclass={'cls_name': 'load_modis'},
    # 0 - get a filename
    _0=init_partial_fn(toolz.identity),
    # 1 - choose image collection
    _1=init_partial_fn(load_fits_to_map),
    # 2 - normalize radius via some resolution
    _2=init_partial_fn(normalize_radius, resolution=resolution, crop = True, padding_factor = 0.1),
    # 3 - correct degregation
    _3=init_partial_fn(correct_degregation, method = "auto"),
    # 4 - sunpy.map.Map --> numpy.ndarray
    _4=init_partial_fn(map_to_ndarray),
    # 5 - Normalize
    _5=init_partial_fn(
        normalize_fn, fn=init_fn(ImageNormalize, vmin=0, vmax=8_800, clip=True, stretch=init_fn(AsinhStretch, a=0.001))),
    # 6 - Reshape (Add batch dimension (?))
    _6=init_partial_fn(
        rearrange, pattern='H W -> H W 1', H=resolution, W=resolution)
)

In [30]:
# choose image collection
collection = ee.ImageCollection("MODIS/006/MOD09A1")

# choose the date
collection = collection.filterDate(, )

In [ ]:
collection.wx.to_xarray(
    path="",
    scale=scale,
    crs=crs,
)

In [20]:
ts = wxee.TimeSeries("MODIS/006/MOD09A1").filterDate("2019-10-01", "2019-11-01")
# ts = ts.select(["sur_refl_b01", "sur_refl_b04", "sur_refl_b03", "sur_refl_b02"])
ts.describe()

MODIS/006/MOD09A1
	Images: 3
	Start date: 2019-10-08 00:00:00 UTC
	End date: 2019-10-24 00:00:00 UTC
	Mean interval: 8.00 days


We need to select a subset region. 
We can use the [Global Administrative Unit Layers 2015](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_SIMPLIFIED_500m_2015_level1#table-schema) which is available within the [`FeatureCollection`](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_SIMPLIFIED_500m_2015_level1#table-schema).

In [25]:
ds = ts.wx.to_xarray(
    region=spain.geometry().bounds(), 
    scale=10_000, 
    crs="EPSG:4326"
)

ds

Requesting data: 100%|████████████████████████████| 3/3 [00:03<00:00,  1.18s/it]


<xarray.Dataset>
Dimensions:          (time: 3, x: 252, y: 181)
Coordinates:
  * time             (time) datetime64[ns] 2019-10-08 2019-10-16 2019-10-24
  * x                (x) float64 -18.19 -18.1 -18.01 ... 4.177 4.267 4.357
  * y                (y) float64 43.79 43.7 43.61 43.52 ... 27.8 27.71 27.62
    spatial_ref      int64 0
Data variables: (12/13)
    sur_refl_b01     (time, y, x) float32 nan nan nan ... 3.865e+03 4.03e+03
    sur_refl_b02     (time, y, x) float32 nan nan nan ... 4.691e+03 4.849e+03
    sur_refl_b03     (time, y, x) float32 nan nan nan ... 1.727e+03 1.838e+03
    sur_refl_b04     (time, y, x) float32 nan nan nan ... 2.74e+03 2.885e+03
    sur_refl_b05     (time, y, x) float32 nan nan nan ... 5.569e+03 5.713e+03
    sur_refl_b06     (time, y, x) float32 nan nan nan ... 5.827e+03 5.97e+03
    ...               ...
    QA               (time, y, x) float64 nan nan nan ... 1.074e+09 1.074e+09
    SolarZenith      (time, y, x) float32 nan nan nan ... 4.406e+03 4.375e+03
    ViewZenith       (time, y, x) float32 nan nan nan ... 1.466e+03 1.253e+03
    RelativeAzimuth  (time, y, x) float32 nan nan nan ... 831.0 4.018e+03
    StateQA          (time, y, x) float64 nan nan nan nan ... 136.0 136.0 136.0
    DayOfYear        (time, y, x) float64 nan nan nan nan ... 302.0 302.0 304.0
Attributes:
    AREA_OR_POINT:           Area
    TIFFTAG_RESOLUTIONUNIT:  1 (unitless)
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    _FillValue:              -32768
    scale_factor:            1.0
    add_offset:              0.0

In [26]:
ds.wx.rgb(stretch=0.98, interactive=True, frame_width=200, aspect=0.5)

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom', widget_type='scrubber')
    [1] WidgetBox(align=('center', 'end'))
        [0] Player(end=2, width=550)

In [ ]:
ds.wx.to